In [1]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 Edg/89.0.774.68" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" "https://storage.googleapis.com/wandb_datasets/nature_12K.zip" -c -O 'nature_12K.zip'

--2021-04-15 06:18:33--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.73.240, 172.253.115.128, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.73.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  33.2MB/s    in 64s     

2021-04-15 06:19:37 (56.5 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]



In [ ]:
!unzip '/content/nature_12K.zip'

In [3]:
import shutil
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPooling2D,Activation,Dropout,Flatten,MaxPool2D
from tensorflow.keras.models import Model,Sequential
import random 
from keras.preprocessing.image import ImageDataGenerator
from keras import applications

In [15]:
class CNN:
  def __init__(self,pretrain=False,pretrained_model="InceptionResNetV2",layers_to_freeze=15,no_of_filters=16,size_of_filters=(3,3)
                ,activation_func="relu",neurons_in_dense_layer=128
               ,dropout=0.3,batch_norm=False,doubling=False,augmentation=False):
    self.augmentation = augmentation
    self.pretrain = pretrain
    self.trained_model = pretrained_model
    self.layers_to_freeze = layers_to_freeze
    self.dropout = dropout
    self.doubling = doubling
    self.batch_norm = batch_norm
    self.no_of_filters = no_of_filters
    self.size_of_filters = size_of_filters
    self.activation_func = activation_func
    self.neurons_in_dense_layer = neurons_in_dense_layer
    self.train_path = "/content/inaturalist_12K/train/"
    self.cv_path = "/content/inaturalist_12K/cv/"
    self.test_path = "/content/inaturalist_12K/val/"
    
  def generate_data(self,img_shape):
    if self.augmentation:
      train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True)
    else:
      train_datagen = ImageDataGenerator(rescale=1./255)
    cv_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    self.train_generator = train_datagen.flow_from_directory(self.train_path,target_size=img_shape,seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")
    self.cv_generator = cv_datagen.flow_from_directory(self.cv_path,target_size=img_shape,seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")
    self.test_generator = test_datagen.flow_from_directory(self.test_path,target_size=img_shape,seed=10,batch_size=64,shuffle=True,class_mode="categorical",color_mode="rgb")

  def create_cv_data(self):
    classes = sorted(os.listdir(self.train_path))[1:]
    os.mkdir(self.cv_path)
    for cls in classes:
      os.mkdir(self.cv_path+cls)

    for cls in classes:
      no_of_files = len(os.listdir(self.train_path+cls))
      files = os.listdir(self.train_path+cls)
      files = np.asarray(files)

      x = [i for i in range(no_of_files)]
      random.shuffle(x)
      index = x[:int(0.1*no_of_files)]
      file_names = files[index]

      for z in file_names:
        shutil.move(self.train_path+cls+"/"+z , self.cv_path+cls+"/"+z)

  def create_model(self):
    if self.pretrain == False:
      self.generate_data((224,224))
      model = Sequential()
      random.seed(10)
      for i in range(5):
        if i==0:
          model.add(Conv2D(self.no_of_filters, self.size_of_filters ,padding="same",activation= self.activation_func, input_shape=(224,224,3)))
        else:
          if self.doubling:
            model.add(Conv2D(self.no_of_filters*(2**i), self.size_of_filters ,padding="same",activation = self.activation_func))
          else:
            model.add(Conv2D(self.no_of_filters, self.size_of_filters ,padding="same",activation = self.activation_func))
        
        if i!=0 and i%2 == 0 and self.batch_norm:
          model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2))) 

      model.add(Flatten())
      model.add(Dropout(self.dropout))
      model.add(Dense(self.neurons_in_dense_layer,activation = self.activation_func))
      if self.batch_norm:
        model.add(BatchNormalization())
      model.add(Dropout(self.dropout))
      model.add(Dense(10,activation='softmax'))

    else:
      self.generate_data((224,224))
      if (self.trained_model=="InceptionV3"):
        pretrained= applications.InceptionV3(include_top=True)
      elif (self.trained_model=="InceptionResNetV2"):
        pretrained= applications.InceptionResNetV2(include_top=True)
      elif (self.trained_model=="ResNet50"):
        pretrained = applications.ResNet50(include_top=True)
      elif (self.trained_model=="Xception"):
        pretrained = applications.Xception(include_top=True)
      elif (self.trained_model=="VGG16"):
        pretrained = applications.VGG16(include_top=True)
      elif (self.trained_model=="VGG19"):
        pretrained= applications.VGG19(include_top=True)
      elif (self.trained_model=="ResNet101"):
        pretrained = applications.ResNet101(include_top=True)
      elif (self.trained_model=="MobileNet"):
        pretrained = applications.MobileNet(include_top=True)
      elif (self.trained_model=="DenseNet201"):
        pretrained= applications.DenseNet201(include_top=True)
      elif (self.trained_model=="NASNetLarge"):
        pretrained= applications.NASNetLarge(include_top=True)
      elif (self.trained_model=="NASNetMobile"):
        pretrained = applications.NASNetMobile(include_top=True)
      x = Dense(10, activation='softmax', name='predictions')(pretrained.layers[-2].output)

      for layer in pretrained.layers[:self.layers_to_freeze]:
        layer.trainable = False

      model = Model(inputs=pretrained.input, outputs=x)

    return model


In [6]:
cnn = CNN()
cnn.create_cv_data()

In [7]:
sweep_config={
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters':{
        'epochs':{
            'values':[3,5,6]
        },
        'layers_to_freeze':{
            'values':[0,8,15,20,30]
        },
        'trained_model':{
           'values':["InceptionV3", "InceptionResNetV2", "ResNet50", "Xception","VGG16","VGG19","ResNet101","MobileNet","DenseNet201","NASNetLarge","NASNetMobile"]
        }
  
    }
}

In [8]:
!pip install --upgrade wandb
import wandb
!wandb login 3c967c63b099a3b2acd600aa30008e7de1ea6498
from wandb.keras import WandbCallback

     |████████████████████████████████| 2.1MB 17.6MB/s 
     |████████████████████████████████| 133kB 56.4MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 163kB 50.9MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=1ddb4a443362341092940cc5b4c6ff2a422e7cf538b8e8d436b214ae599431df
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=bf5608b7e55e67f4e9a83548594c5a605dbaf172efe8bfc05927bdad63eef560
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
sweep_id = wandb.sweep(sweep_config,project="finetuning pretrained model", entity="adi00510")

Create sweep with ID: ru1t705e
Sweep URL: https://wandb.ai/adi00510/finetuning%20pretrained%20model/sweeps/ru1t705e


In [13]:
def train():
    config_defaults={
      'epochs':5,
      'trained_model':"InceptionV3",
      'layers_to_freeze':15
       }
    
    wandb.init(config=config_defaults)
    config=wandb.config
    
    cnn = CNN(pretrain=True,pretrained_model=config.trained_model,layers_to_freeze=config.layers_to_freeze)
    model = cnn.create_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(cnn.train_generator, validation_data = cnn.cv_generator,epochs=config.epochs,callbacks=[WandbCallback()])



In [ ]:
wandb.agent(sweep_id,train)